In [1]:
cd ..

/Users/snow/Documents/wd/TCR-DeepInsight


In [14]:
from tcr_deep_insight.model.modeling_bert._model import (
    TRabModelingBertForPseudoSequence,
    TCRpMHCPairAttention
)
from tcr_deep_insight.model.tokenizers._tokenizer import (
    tokenize_tcr_pseudo_sequence_to_fixed_length, 
    trab_tokenizer_for_pseudosequence,
    tokenize_to_fixed_length
)
from tcr_deep_insight.model.modeling_bert._collator import AminoAcidsCollator
from tcr_deep_insight.model.modeling_bert._config import get_human_config
from tcr_deep_insight.utils._tcr_definitions import TCR

config = get_human_config(bert_type="tiny", vocab_size=36)

In [3]:
df = pd.read_parquet(
    "/Volumes/rsch/TCRFoldAndDock/TCRGen//merged_clean_trb.2.pseudosequence.mait.parquet"
)

In [ ]:
import torch
sequence = df["pseudosequence"][0]
sequence, input_ids, attention_mask = tokenize_tcr_pseudo_sequence_to_fixed_length(sequence)
input_ids = np.array(input_ids)
attention_mask = np.array(attention_mask)


In [4]:
model = TRabModelingBertForPseudoSequence(bert_config=config)

In [5]:
collator = AminoAcidsCollator(
    mask_token_id=4,
    max_length=100,
    mlm_probability=0.15,
)

In [ ]:
input_ids_mlm, attention_mask_mlm = collator(input_ids[None,:], attention_mask[None,:])
input_ids_mlm = torch.tensor(input_ids_mlm)
attention_mask_mlm = torch.tensor(attention_mask_mlm)
input_ids = torch.tensor(input_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [7]:
import pickle
with open("/Volumes/rsch/TCRFoldAndDock/TCRFold/processed_pairing_data/6amu_human_tcr_only_6amu_human_pmhc_only_A02:01.pkl", "rb") as f:
    data = pickle.load(f)

In [13]:
TCR(data['tcr_sequence'])

AttributeError: 'TCR' object has no attribute 'alpha_j_motif'

In [16]:
from tcr_deep_insight.utils._tcr_definitions import (
    _get_hla_pseudo_sequence,
    blosum_align,
    HLA_I_PSEUDO_INDEX,
)

import pickle
for i in os.listdir("/Volumes/rsch/TCRFoldAndDock/TCRFold/processed_pairing_data/"):
    with open("/Volumes/rsch/TCRFoldAndDock/TCRFold/processed_pairing_data/" + i, "rb") as f:
        data = pickle.load(f)

    if 'tcr_pmhc_pseudo_distogram' in data.keys():
        if data['tcr_pmhc_pseudo_distogram'].shape[1] == 110:
            continue

    hla_pseudo_sequence = _get_hla_pseudo_sequence()

    try:
        alignment = blosum_align(
            hla_pseudo_sequence["A*02:01"]["fullsequence"], data["pmhc_sequence"]
        )
        indices = [alignment[i] for i in HLA_I_PSEUDO_INDEX]
    except:
        print(i)
        continue

    tcr_pmhc_distogram = np.zeros((110, len(HLA_I_PSEUDO_INDEX) + 17, 64))

    indices = [alignment[i] for i in HLA_I_PSEUDO_INDEX]

    distogram = data["cdr1a_mhc_ca_distogram"][:, list(indices)]
    tcr_pmhc_distogram[
        1 + 0 : 1 + distogram.shape[0], 
        1 + 0 : 1 + distogram.shape[1]
    ] = distogram

    distogram = data["cdr2a_mhc_ca_distogram"][:, list(indices)]
    tcr_pmhc_distogram[
        1 + 8 + 1 : 1 + 8 + 1 + distogram.shape[0], 
        1 + 0 : 1 + distogram.shape[1]
    ] = distogram

    distogram = data["cdr3a_mhc_ca_distogram"][:, list(indices)]
    tcr_pmhc_distogram[
        1 + 16 + 2 : 1 + 16 + 2 + distogram.shape[0], 
        1 + 0 : 1 + distogram.shape[1]
    ] = data["cdr3a_mhc_ca_distogram"][:, list(indices)]

    distogram = data["cdr1b_mhc_ca_distogram"][:, list(indices)]
    tcr_pmhc_distogram[
        1 + 52 + 3 : 1 + 52 + 3 + distogram.shape[0], 
        1 + 0 : 1 + distogram.shape[1]
    ] = distogram

    distogram = data["cdr2b_mhc_ca_distogram"][:, list(indices)]
    tcr_pmhc_distogram[
        1 + 60 + 4 : 1 + 60 + 4 + distogram.shape[0], 
        1 + 0 : 1 + distogram.shape[1]
    ] = distogram

    distogram = data["cdr3b_mhc_ca_distogram"][:, list(indices)]
    tcr_pmhc_distogram[
        1 + 68 + 5 : 1 + 68 + 5 + distogram.shape[0], 
        1 + 0 : 1 + distogram.shape[1]
    ] = data["cdr3b_mhc_ca_distogram"][:, list(indices)]

    distogram = data["cdr1a_peptide_ca_distogram"]
    tcr_pmhc_distogram[
        1 + 0 : 1 + distogram.shape[0],
        1 + len(HLA_I_PSEUDO_INDEX) + 1 : 1 + len(HLA_I_PSEUDO_INDEX) + 1 + distogram.shape[1],
    ] = distogram

    distogram = data["cdr2a_peptide_ca_distogram"]
    tcr_pmhc_distogram[
        1 + 8 + 1 : 1 + 8 + 1 + distogram.shape[0],
        1 + len(HLA_I_PSEUDO_INDEX) + 1 : 1 + len(HLA_I_PSEUDO_INDEX) + 1 + distogram.shape[1],
    ] = distogram

    distogram = data["cdr3a_peptide_ca_distogram"]
    tcr_pmhc_distogram[
        1 + 16 + 2 : 1 + 16 + 2 + distogram.shape[0],
        1 + len(HLA_I_PSEUDO_INDEX) + 1 : 1 + len(HLA_I_PSEUDO_INDEX) + 1 + distogram.shape[1],
    ] = data["cdr3a_peptide_ca_distogram"]

    distogram = data["cdr1b_peptide_ca_distogram"]
    tcr_pmhc_distogram[
        1 + 52 + 3 : 1 + 52 + 3 + distogram.shape[0],
        1 + len(HLA_I_PSEUDO_INDEX) + 1 : 1 + len(HLA_I_PSEUDO_INDEX) + 1 + distogram.shape[1],
    ] = distogram

    distogram = data["cdr2b_peptide_ca_distogram"]
    tcr_pmhc_distogram[
        1 + 60 + 4 : 1 + 60 + 4 + distogram.shape[0],
        1 + len(HLA_I_PSEUDO_INDEX) + 1 : 1 + len(HLA_I_PSEUDO_INDEX) + 1 + distogram.shape[1],
    ] = distogram

    distogram = data["cdr3b_peptide_ca_distogram"]
    tcr_pmhc_distogram[
        1 + 68 + 5 : 1 + 68 + 5 + distogram.shape[0],
        1 + len(HLA_I_PSEUDO_INDEX) + 1 : 1 + len(HLA_I_PSEUDO_INDEX) + 1 + distogram.shape[1],
    ] = data["cdr3b_peptide_ca_distogram"]

    data['tcr_pmhc_pseudo_distogram'] = tcr_pmhc_distogram

    with open("/Volumes/rsch/TCRFoldAndDock/TCRFold/processed_pairing_data/" + i, "wb") as f:
        pickle.dump(data, f)




4grl_human_tcr_only_4grl_human_pmhc_only_DQA1-01_DQB1-05.pkl
4ozh_human_tcr_only_4ozh_human_pmhc_only_DQA1-05_DQB1-02.pkl
5ksb_human_tcr_only_5ksb_human_pmhc_only_DQA1-05_DQB1-03.pkl
4ozi_human_tcr_only_4ozi_human_pmhc_only_DQA1-05_DQB1-02.pkl
5ks9_human_tcr_only_5ks9_human_pmhc_only_DQA1-03_DQB1-03.pkl
6py2_human_tcr_only_6py2_human_pmhc_only_DQA1-02_DQB1-02.pkl
4z7w_human_tcr_only_4z7w_human_pmhc_only_DQA1-03_DQB1-03.pkl
4z7v_human_tcr_only_4z7v_human_pmhc_only_DQA1-03_DQB1-03.pkl
4may_human_tcr_only_4may_human_pmhc_only_DQA1-01_DQB1-05.pkl
4ozg_human_tcr_only_4ozg_human_pmhc_only_DQA1-05_DQB1-02.pkl
4ozf_human_tcr_only_4ozf_human_pmhc_only_DQA1-05_DQB1-02.pkl
3pl6_human_tcr_only_3pl6_human_pmhc_only_DQA1-01_DQB1-05.pkl
4z7u_human_tcr_only_4z7u_human_pmhc_only_DQA1-03_DQB1-03.pkl
6dfx_human_tcr_only_6dfx_human_pmhc_only_DQA1-03_DQB1-03.pkl
6u3o_human_tcr_only_6u3o_human_pmhc_only_DQA1-05_DQB1-02.pkl
6u3n_human_tcr_only_6u3n_human_pmhc_only_DQA1-05_DQB1-02.pkl
6px6_human_tcr_only_6px6

In [13]:
data['tcr_pmhc_pseudo_distogram'].shape

KeyError: 'tcr_pmhc_pseudo_distogram'

In [6]:
import pickle
import numpy as np 
from tcr_deep_insight.utils._tcr_definitions import (
    _get_hla_pseudo_sequence,
    blosum_align,
    HLA_I_PSEUDO_INDEX,
)

with open("/Volumes/rsch/TCRFoldAndDock/TCRFold/processed_pairing_data/1ao7_human_tcr_only_1ao7_human_pmhc_only_A02:01.pkl", "rb") as f:
    data = pickle.load(f)

hla_pseudo_sequence = _get_hla_pseudo_sequence()

pmhc_pseudoseq = hla_pseudo_sequence["A*02:01"]["pseudosequence"] + ':' + data['peptide']
tcr_pseudoseq = ':'.join([
    data[k] for k in ['cdr1a',  'cdr2a', 'cdr3a', 'cdr1b','cdr2b', 'cdr3b']
])

tcr_sequence, tcr_input_ids, tcr_attention_mask = tokenize_tcr_pseudo_sequence_to_fixed_length(tcr_pseudoseq)
tcr_input_ids = np.array(tcr_input_ids)
tcr_attention_mask = np.array(tcr_attention_mask)

pmhc_sequence, pmhc_input_ids, pmhc_attention_mask = tokenize_to_fixed_length(pmhc_pseudoseq, 50)
pmhc_input_ids = np.array(pmhc_input_ids)
pmhc_attention_mask = np.array(pmhc_attention_mask)

In [15]:
import torch
pad_token = '.'
max_length = {
    'CDR3a': 36,
    'CDR3b': 36,
    'CDR1b': 6,
    'CDR2b': 8,
    'CDR1a': 6,
    'CDR2a': 8,
}
cdr1a, cdr2a, cdr3a, cdr1b, cdr2b, cdr3b = tcr_pseudoseq.split(':')

cdr1a = cdr1a + pad_token * (max_length['CDR1a'] - len(cdr1a))
cdr2a = cdr2a + pad_token * (max_length['CDR2a'] - len(cdr2a))
cdr1b = cdr1b + pad_token * (max_length['CDR1b'] - len(cdr1b))
cdr2b = cdr2b + pad_token * (max_length['CDR2b'] - len(cdr2b))
cdr3a = cdr3a + pad_token * (max_length['CDR3a'] - len(cdr3a))
cdr3b = cdr3b + pad_token * (max_length['CDR3b'] - len(cdr3b))

tcr_pmhc_distogram_df = pd.DataFrame(
    torch.tensor(tcr_pmhc_distogram).topk(1)[1].squeeze().numpy(),
    index=list("^" + ":".join([cdr1a, cdr2a, cdr3a, cdr1b, cdr2b, cdr3b])),
    columns=list(pmhc_sequence),
)

sns.heatmap(tcr_pmhc_distogram_df)

'^YFAMYGEKVAHTHVDTLYVRYHYYTWAVLAYTWY:EDFVYQFKGMCYFTNGTERVRLVTRYIYNREEYARFDSDVGVYRAVTPLGPPAAEYWNSQKEVLERTRAELDTVCRHNYQLELRTTLQRGEGSFQPSQEN'

In [7]:
import torch
head = TCRpMHCPairAttention(config)

tcr_input_ids = torch.from_numpy(tcr_input_ids).unsqueeze(0)
tcr_attention_mask = torch.from_numpy(tcr_attention_mask).unsqueeze(0).to(torch.float32)

tcr_output = model(
    input_ids=tcr_input_ids,
    attention_mask=tcr_attention_mask,
    labels=tcr_input_ids
)

tcr_hidden_states = tcr_output["output"].hidden_states[-1]

In [8]:
self=head

pmhc_input_ids = torch.from_numpy(pmhc_input_ids).unsqueeze(0)
pmhc_attention_mask = torch.from_numpy(pmhc_attention_mask).unsqueeze(0).to(torch.float32)

output = self.model.forward(
    input_ids=pmhc_input_ids,
    attention_mask=pmhc_attention_mask.to(torch.float32),
    return_dict=True,
    output_hidden_states=True,
)

pmhc_hidden_states = output['hidden_states'][-1]

In [9]:
z, z_mask = self.embedder(
    tcr_hidden_states,
    pmhc_hidden_states,
    tcr_attention_mask.to(torch.float32),
    pmhc_attention_mask.to(torch.float32)
)
z.shape, z_mask.shape

(torch.Size([1, 106, 51, 128]), torch.Size([1, 106, 51]))

In [10]:
tcr_hidden_states.shape, pmhc_hidden_states.shape, tcr_attention_mask.shape, pmhc_attention_mask.shape, z.shape

(torch.Size([1, 106, 128]),
 torch.Size([1, 51, 128]),
 torch.Size([1, 106]),
 torch.Size([1, 51]),
 torch.Size([1, 106, 51, 128]))

In [11]:
self.layers[0](
    z, 
    tcr_hidden_states, 
    pmhc_hidden_states, 
    tcr_attention_mask, 
    pmhc_attention_mask
)

EinopsError:  Error while processing rearrange-reduction pattern "batch h c k -> batch h k c".
 Input tensor shape: torch.Size([1, 106, 2, 51, 128]). Additional info: {}.
 Expected 4 dimensions, got 5

In [31]:
self.layers[0].tri_att_start.mha(z, z)

EinopsError:  Error while processing rearrange-reduction pattern "b q (h c) -> b h q c".
 Input tensor shape: torch.Size([1, 106, 51, 256]). Additional info: {'h': 2}.
 Expected 3 dimensions, got 4